In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import xgboost
import sklearn.metrics as metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, classification_report, confusion_matrix
from sklearn import preprocessing
from math import sqrt

In [2]:
col=['age','workclass','fnlwgt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','income']
path = './adult.data'
df = pd.read_csv(path, header=None, sep=',\s')
df.columns=col

path2 = './adult.test'
df2 = pd.read_csv(path2, header=None, sep=',\s')
df2.columns=col

c:\users\user\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\user\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


In [3]:
# workclass:政府、軍隊、私人
# fnlwgt:人口普查員ID
# education:教育水平
# education_num:education轉數字
# marital_status:婚姻狀況
# occupation:工作類型
# relationship:家庭
# race:種族
# capital_gain:資本利益
# capital_loss:資本損失
# hours_per_week:一週工時
# native_country:國籍
# income:收入範圍

In [4]:
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
df2.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [6]:
#workclass, occupation, native_country have " ?" value
df=df.replace(to_replace=r'^ ?.$', value=np.nan, regex=True)
df2=df2.replace(to_replace=r'^ ?.$', value=np.nan, regex=True)

In [7]:
df=df.dropna()
df2=df2.dropna()

In [8]:
labelencoder = LabelEncoder()
df['workclass'] = labelencoder.fit_transform(df['workclass'])
df['marital_status'] = labelencoder.fit_transform(df['marital_status'])
df['occupation'] = labelencoder.fit_transform(df['occupation'])
df['relationship'] = labelencoder.fit_transform(df['relationship'])
df['race'] = labelencoder.fit_transform(df['race'])
df['sex'] = labelencoder.fit_transform(df['sex'])
df['native_country'] = labelencoder.fit_transform(df['native_country'])
df['income'] = labelencoder.fit_transform(df['income'])
y_train=df['hours_per_week']
df.drop(columns=['fnlwgt','education','hours_per_week'],inplace=True)
df.head()

,age,workclass,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,native_country,income
0,39,5,13,4,0,1,4,1,2174,0,38,0
1,50,4,13,2,3,0,4,1,0,0,38,0
2,38,2,9,0,5,1,4,1,0,0,38,0
3,53,2,7,2,5,0,2,1,0,0,38,0
4,28,2,13,2,9,5,2,0,0,0,4,0


In [9]:
df2['workclass'] = labelencoder.fit_transform(df2['workclass'])
df2['marital_status'] = labelencoder.fit_transform(df2['marital_status'])
df2['occupation'] = labelencoder.fit_transform(df2['occupation'])
df2['relationship'] = labelencoder.fit_transform(df2['relationship'])
df2['race'] = labelencoder.fit_transform(df2['race'])
df2['sex'] = labelencoder.fit_transform(df2['sex'])
df2['native_country'] = labelencoder.fit_transform(df2['native_country'])
df2['income'] = labelencoder.fit_transform(df2['income'])
y_test=df2['hours_per_week']
df2.drop(columns=['fnlwgt','education','hours_per_week'],inplace=True)
df2.head()

,age,workclass,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,native_country,income
0,25,2,7,4,6,3,2,1,0,0,37,0
1,38,2,9,2,4,0,4,1,0,0,37,0
2,28,1,12,2,10,0,4,1,0,0,37,1
3,44,2,10,2,6,0,2,1,7688,0,37,1
5,34,2,6,4,7,1,4,1,0,0,37,0


In [10]:
from sklearn.feature_selection import SelectKBest, f_regression
selector = SelectKBest(f_regression,k='all').fit(df,y_train)

In [11]:
selector_score=selector.scores_
connect={"Columns": df.columns, "SK_Score": selector_score}
pd.DataFrame(connect).sort_values(by=['SK_Score'], ascending=False)

,Columns,SK_Score
5,relationship,2148.055645
7,sex,1704.258147
11,income,1676.548626
3,marital_status,1117.294306
2,education_num,718.321849
0,age,314.567872
8,capital_gain,196.383781
9,capital_loss,83.094325
1,workclass,77.798247
6,race,71.204917


In [12]:
df.drop(columns=['native_country'],inplace=True)
df2.drop(columns=['native_country'],inplace=True)

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df)
X_scaled = scaler.transform(df)
scaler2 = StandardScaler().fit(df2)
X_scaled2 = scaler2.transform(df2)

In [14]:
knn = KNeighborsRegressor(n_neighbors=25, p=1, weights='uniform')
knn.fit(X_scaled,y_train)

KNeighborsRegressor(n_neighbors=25, p=1)

In [15]:
y_pred=knn.predict(X_scaled2)
print("MAE:",mean_absolute_error(y_test,y_pred))
print("RMSE:",sqrt(mean_squared_error(y_test,y_pred)))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred))

MAE: 7.210974767596282
RMSE: 10.55767578752675
MAPE: 0.27101461294485973


In [16]:
svr = SVR(kernel='rbf',C=45, epsilon=0.2, gamma='scale')
svr.fit(X_scaled,y_train)

SVR(C=45, epsilon=0.2)

In [17]:
y_pred=svr.predict(X_scaled2)
print("MAE:",mean_absolute_error(y_test,y_pred))
print("RMSE:",sqrt(mean_squared_error(y_test,y_pred)))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred))

MAE: 6.775074959401162
RMSE: 10.797388738285251
MAPE: 0.2611527261942778


In [29]:
rf = RandomForestRegressor(min_samples_split=80, n_estimators=1240)
rf.fit(X_scaled,y_train)

RandomForestRegressor(min_samples_split=80, n_estimators=1240)

In [30]:
y_pred=rf.predict(X_scaled2)
print("MAE:",mean_absolute_error(y_test,y_pred))
print("RMSE:",sqrt(mean_squared_error(y_test,y_pred)))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred))

MAE: 7.018671519860376
RMSE: 10.35370559310268
MAPE: 0.26081230259723065


In [21]:
xg=xgboost.XGBRegressor(n_estimators=184, max_depth=5, min_child_weight=9, gamma=0.01,colsample_bytree=0.8
                       , reg_alpha=0.05, reg_lambda=3, learning_rate=0.02)
xg.fit(X_scaled,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0.01, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.02, max_delta_step=0,
             max_depth=5, min_child_weight=9, missing=nan,
             monotone_constraints='()', n_estimators=184, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0.05, reg_lambda=3, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [22]:
y_pred=xg.predict(X_scaled2)
print("MAE:",mean_absolute_error(y_test,y_pred))
print("RMSE:",sqrt(mean_squared_error(y_test,y_pred)))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred))

MAE: 6.9154317720319485
RMSE: 10.407719012373144
MAPE: 0.2549912626628526
